In [1]:
import torch
from torch import nn as nn
import torchvision
from torch.utils.data import DataLoader, dataset
from torchvision.transforms import AutoAugment, Normalize, Compose, ToTensor
import torch.nn.functional as F
from torchmetrics import Accuracy
import numpy as np

In [2]:
class accuracy:
    def __init__(self):
        pass
    def update(self, x,y):
        pass
    def reset(self):
        pass

In [3]:
def __normalize__(x):
    return x/255.
normalize = torch.jit.script(__normalize__)

In [4]:
from torchvision.datasets import CIFAR100

In [5]:
data = CIFAR100("./data", train = True, download = True, transform = Compose([AutoAugment(),ToTensor(), normalize]))
test_data = CIFAR100("./data", train = False, download = True, transform = Compose([AutoAugment(),ToTensor(),  normalize]))

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_set = DataLoader(data, 128, shuffle = True, num_workers=2)
test_set = DataLoader(test_data, 128, shuffle = False, num_workers=1)

In [46]:
class VGG5(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq_1 = nn.Sequential(nn.Conv2d(3, 64, 2,2), nn.GELU(),                                                                     
                                   nn.BatchNorm2d(64))
        self.seq_2 = nn.Sequential(nn.Conv2d(64, 128, 2,1), nn.GELU(),
                                   nn.Conv2d(128, 128, 2,1), nn.GELU(), 
                                   nn.Conv2d(128, 128, 2,1), nn.GELU(), 
                                   nn.MaxPool2d(2,2, ), 
                                   nn.BatchNorm2d(128))
        self.seq_3 = nn.Sequential(nn.Conv2d(128, 256, 2,1, padding = "same"), nn.GELU(),
                                   nn.Conv2d(256, 256, 2,1, padding = "same"), nn.GELU(), 
                                   nn.Conv2d(256, 256, 2,1, padding = "same"), nn.GELU(), 
                                   nn.MaxPool2d(2,1), 
                                   nn.BatchNorm2d(256, ))
        self.seq_4 = nn.Sequential(nn.Conv2d(256, 512, 2,1, padding = "same"), nn.GELU(),
                                   nn.Conv2d(512, 512, 2,1, padding = "same"), nn.GELU(), 
                                   nn.Conv2d(512, 512, 2,1, padding = "same"), nn.GELU(), 
                                   nn.MaxPool2d(2,1), 
                                   nn.BatchNorm2d(512, ))
        self.f = nn.Flatten()
        self.dense = nn.Linear(8192, 100)
    def forward(self, x):
        x = self.seq_1(x)
        x = self.seq_2(x)
        x = self.seq_3(x)
        x = self.seq_4(x)
        x = self.f(x)
        x = self.dense(x)
        return x

In [47]:
model_ = VGG5()
model = torch.nn.DataParallel(model_)
model.to(0)


DataParallel(
  (module): VGG5(
    (seq_1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(2, 2), stride=(2, 2))
      (1): GELU(approximate='none')
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
      (1): GELU(approximate='none')
      (2): Conv2d(128, 128, kernel_size=(2, 2), stride=(1, 1))
      (3): GELU(approximate='none')
      (4): Conv2d(128, 128, kernel_size=(2, 2), stride=(1, 1))
      (5): GELU(approximate='none')
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (1): GELU(approximate='none')
      (2): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (3): GELU(approximate='n

In [48]:
model(torch.randn(1,3,32,32)).shape

torch.Size([1, 100])

In [49]:
def loss(x,y):
    return nn.CrossEntropyLoss()(model(x),y)

In [50]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)

In [ ]:
t = 0
while True:
    L = []
    loss__ = []
    model.train()

    for i, batch in enumerate(train_set):
        x, y = batch[0].cuda(), batch[1].cuda()
    
        loss_ = loss(x,y)
        loss_.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss__.append(loss_.item())

    model.eval()
    with torch.no_grad():
            
        for j, batch in enumerate(test_set):
            cond = 1
            if cond: ### here we do reservoir sampling
                x, y = batch[0].cuda(), batch[1].cuda()
                L.append(sum(torch.argmax(model(x), dim = 1) == y).item())
            else:
                continue
        
    print(t, sum(L)/len(L), sum(loss__)/len(loss__))
    t += 1

0 10.531645569620252 4.372416216394176
1 14.962025316455696 4.065877507104898
2 17.59493670886076 3.882354469250535


In [ ]:
np.random.rand()